In [ ]:
import os
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.document_loaders import DirectoryLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Pinecone as LangPicone
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain
from pinecone import Pinecone as PineconeClient

In [ ]:
#initialize openai and pinecone keys
openai_key = ""
pinecone_key = ""
pinecone_env = ""
pinecone_index = ""
os.environ['PINECONE_API_KEY'] = pinecone_key
os.environ["OPENAI_API_KEY"] = openai_key

In [ ]:
# Initialize HYPERPARAMETERS

chunck_size = 1000 # commonly used -> 1000
chunck_overlap = 0

In [ ]:
# load the docs
dic = 'path/to/docs'
docs = DirectoryLoader(dic).load()

In [ ]:
# convert the doc to list
docs_list = [d.page_content for d in docs]
docs_list

In [ ]:
# Split the docs into small chunks
# Tech used -> recusrsive splitting

splitter = RecursiveCharacterTextSplitter(chunk_size=chunck_size, chunk_overlap=chunck_overlap)
split_docs = splitter.create_documents(docs_list)
split_docs_list = [d.page_content for d in split_docs]
split_docs_list

In [ ]:
# get the embeddings of the chunks
embdClient = OpenAIEmbeddings(openai_api_key=openai_key)
docEmbd = LangPicone.from_texts(split_docs_list,embdClient, index_name=pinecone_index)

In [ ]:
qn = input('\nQuestion:\t')

relevantDoc_score = docEmbd.similarity_search_with_score(qn)
relevantDoc = [d[0] for d in relevantDoc_score]

model = OpenAI(temperature=0.5)
chain = load_qa_chain(model,chain_type='stuff')

ans = chain.run(input_documents=relevantDoc,question=qn)
print(f'\nqn:\t{qn}\nAnswer:\t{ans}')

top_3_chunks = []
print('\nTop 3 chunks (stored in top_3_chunks)\nScore\tChunk\n')
for i in range(3):
    print(f'Score:\t{relevantDoc_score[i][1]}\nChunk:\t{relevantDoc_score[i][0].page_content}\n')
    top_3_chunks.append(relevantDoc_score[i][0].page_content)